# Data Aggregation and Group Operations

In [2]:
import pandas as pd 
import numpy as np

## GroupBy mechanics

In [6]:
df = pd.DataFrame({'key1' : list('aabba'),
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                  'balance' : np.random.randn(5) * 10,
                   'income' : np.random.randn(5) + 2
                  })

df

balance    income key1 key2
0  -3.570696  0.608132    a  one
1  12.863333  3.596167    a  two
2   5.253448  1.454775    b  one
3   6.851577  2.950287    b  two
4  -3.649933  4.120578    a  one

In [7]:
df.mean()

balance    3.549546
income     2.545988
dtype: float64

In [14]:
means = df.groupby('key1').mean()
means

balance    income
key1                    
a     1.880901  2.774959
b     6.052513  2.202531

In [12]:
mean_key1 = df.groupby('key1')['balance'].mean()
mean_key1

key1
a    1.880901
b    6.052513
Name: balance, dtype: float64

In [13]:
mean_key1['a']

1.8809013841247915

In [17]:
means

balance    income
key1                    
a     1.880901  2.774959
b     6.052513  2.202531

In [16]:
means['balance']['a']

1.8809013841247915

In [19]:
df.groupby(['key1', 'key2']).mean()

balance    income
key1 key2                     
a    one   -3.610314  2.364355
     two   12.863333  3.596167
b    one    5.253448  1.454775
     two    6.851577  2.950287

In [22]:
df.groupby(['key1', 'key2']).agg(['mean', 'count'])

balance          income      
                mean count      mean count
key1 key2                                 
a    one   -3.610314     2  2.364355     2
     two   12.863333     1  3.596167     1
b    one    5.253448     1  1.454775     1
     two    6.851577     1  2.950287     1

In [28]:
df

balance    income key1 key2
0  -3.570696  0.608132    a  one
1  12.863333  3.596167    a  two
2   5.253448  1.454775    b  one
3   6.851577  2.950287    b  two
4  -3.649933  4.120578    a  one

We can provide arbitrary functions to the .agg() method of groupby objects:

In [30]:
df.groupby('key1')['key2'].agg(lambda strseries: strseries.str.len().sum())

key1
a    9
b    6
Name: key2, dtype: int64

### Iterating over groups

In [34]:
for value, groupmembers in df.groupby('key1'):
    print(value)
    print(groupmembers)

a
     balance    income key1 key2
0  -3.570696  0.608132    a  one
1  12.863333  3.596167    a  two
4  -3.649933  4.120578    a  one
b
    balance    income key1 key2
2  5.253448  1.454775    b  one
3  6.851577  2.950287    b  two


In [39]:
list(df.groupby('key1'))

[('a',      balance    income key1 key2
  0  -3.570696  0.608132    a  one
  1  12.863333  3.596167    a  two
  4  -3.649933  4.120578    a  one), ('b',     balance    income key1 key2
  2  5.253448  1.454775    b  one
  3  6.851577  2.950287    b  two)]

In [40]:
dict(list(df.groupby('key1')))

{'a':      balance    income key1 key2
 0  -3.570696  0.608132    a  one
 1  12.863333  3.596167    a  two
 4  -3.649933  4.120578    a  one, 'b':     balance    income key1 key2
 2  5.253448  1.454775    b  one
 3  6.851577  2.950287    b  two}

## Data aggregation

In [42]:
import requests

url = 'https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv'
response = requests.get(url)

out_file = open('tips.csv', 'wb')
out_file.write(response.content)
out_file.close()

In [43]:
tips = pd.read_csv('tips.csv')
tips.head()

total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4

#### Exercise

Extract the amount of tip expressed as % of the bill. Compare the average value for men and women. Are they substatially different? What values would you need to calculate a p-value, assuming a normal distribution?

In [45]:
tips['tip_pct'] = 100 * tips['tip'] / tips['total_bill']
tips.head()

total_bill   tip     sex smoker  day    time  size    tip_pct
0       16.99  1.01  Female     No  Sun  Dinner     2   5.944673
1       10.34  1.66    Male     No  Sun  Dinner     3  16.054159
2       21.01  3.50    Male     No  Sun  Dinner     3  16.658734
3       23.68  3.31    Male     No  Sun  Dinner     2  13.978041
4       24.59  3.61  Female     No  Sun  Dinner     4  14.680765

In [47]:
tips.groupby('sex')['tip_pct'].mean()

sex
Female    16.649074
Male      15.765055
Name: tip_pct, dtype: float64

In [49]:
tips.groupby('sex')['tip_pct'].agg(['mean', 'std', 'count'])

mean       std  count
sex                               
Female  16.649074  5.363173     87
Male    15.765055  6.477787    157

In [54]:
stacked = df.groupby(['key1', 'key2']).mean()
stacked

balance    income
key1 key2                     
a    one   -3.610314  2.364355
     two   12.863333  3.596167
b    one    5.253448  1.454775
     two    6.851577  2.950287

In [56]:
stacked.unstack('key1')

balance              income          
key1          a         b         a         b
key2                                         
one   -3.610314  5.253448  2.364355  1.454775
two   12.863333  6.851577  3.596167  2.950287

In [59]:
stacked.unstack('key1').unstack('balance')

         key1  key2
balance  a     one     -3.610314
               two     12.863333
         b     one      5.253448
               two      6.851577
income   a     one      2.364355
               two      3.596167
         b     one      1.454775
               two      2.950287
dtype: float64

In [66]:
tips.pivot(columns='sex')

total_bill           tip       smoker          day          time          \
sex     Female   Male Female  Male Female  Male Female  Male  Female    Male   
0        16.99    NaN   1.01   NaN     No  None    Sun  None  Dinner    None   
1          NaN  10.34    NaN  1.66   None    No   None   Sun    None  Dinner   
2          NaN  21.01    NaN  3.50   None    No   None   Sun    None  Dinner   
3          NaN  23.68    NaN  3.31   None    No   None   Sun    None  Dinner   
4        24.59    NaN   3.61   NaN     No  None    Sun  None  Dinner    None   
5          NaN  25.29    NaN  4.71   None    No   None   Sun    None  Dinner   
6          NaN   8.77    NaN  2.00   None    No   None   Sun    None  Dinner   
7          NaN  26.88    NaN  3.12   None    No   None   Sun    None  Dinner   
8          NaN  15.04    NaN  1.96   None    No   None   Sun    None  Dinner   
9          NaN  14.78    NaN  3.23   None    No   None   Sun    None  Dinner   
10         NaN  10.27    NaN  1.71   None    No   None   Sun    None  Dinner   
11       35.26    NaN   5.00   NaN     No  None    Sun  None  Dinner    None   
12         NaN  15.42    NaN  1.57   None    No   None   Sun    None  Dinner   
13         NaN  18.43    NaN  3.00   None    No   None   Sun    None  Dinner   
14       14.83    NaN   3.02   NaN     No  None    Sun  None  Dinner    None   
15         NaN  21.58    NaN  3.92   None    No   None   Sun    None  Dinner   
16       10.33    NaN   1.67   NaN     No  None    Sun  None  Dinner    None   
17         NaN  16.29    NaN  3.71   None    No   None   Sun    None  Dinner   
18       16.97    NaN   3.50   NaN     No  None    Sun  None  Dinner    None   
19         NaN  20.65    NaN  3.35   None    No   None   Sat    None  Dinner   
20         NaN  17.92    NaN  4.08   None    No   None   Sat    None  Dinner   
21       20.29    NaN   2.75   NaN     No  None    Sat  None  Dinner    None   
22       15.77    NaN   2.23   NaN     No  None    Sat  None  Dinner    None   
23         NaN  39.42    NaN  7.58   None    No   None   Sat    None  Dinner   
24         NaN  19.82    NaN  3.18   None    No   None   Sat    None  Dinner   
25         NaN  17.81    NaN  2.34   None    No   None   Sat    None  Dinner   
26         NaN  13.37    NaN  2.00   None    No   None   Sat    None  Dinner   
27         NaN  12.69    NaN  2.00   None    No   None   Sat    None  Dinner   
28         NaN  21.70    NaN  4.30   None    No   None   Sat    None  Dinner   
29       19.65    NaN   3.00   NaN     No  None    Sat  None  Dinner    None   
..         ...    ...    ...   ...    ...   ...    ...   ...     ...     ...   
214      28.17    NaN   6.50   NaN    Yes  None    Sat  None  Dinner    None   
215      12.90    NaN   1.10   NaN    Yes  None    Sat  None  Dinner    None   
216        NaN  28.15    NaN  3.00   None   Yes   None   Sat    None  Dinner   
217        NaN  11.59    NaN  1.50   None   Yes   None   Sat    None  Dinner   
218        NaN   7.74    NaN  1.44   None   Yes   None   Sat    None  Dinner   
219      30.14    NaN   3.09   NaN    Yes  None    Sat  None  Dinner    None   
220        NaN  12.16    NaN  2.20   None   Yes   None   Fri    None   Lunch   
221      13.42    NaN   3.48   NaN    Yes  None    Fri  None   Lunch    None   
222        NaN   8.58    NaN  1.92   None   Yes   None   Fri    None   Lunch   
223      15.98    NaN   3.00   NaN     No  None    Fri  None   Lunch    None   
224        NaN  13.42    NaN  1.58   None   Yes   None   Fri    None   Lunch   
225      16.27    NaN   2.50   NaN    Yes  None    Fri  None   Lunch    None   
226      10.09    NaN   2.00   NaN    Yes  None    Fri  None   Lunch    None   
227        NaN  20.45    NaN  3.00   None    No   None   Sat    None  Dinner   
228        NaN  13.28    NaN  2.72   None    No   None   Sat    None  Dinner   
229      22.12    NaN   2.88   NaN    Yes  None    Sat  None  Dinner    None   
230        NaN  24.01    NaN  2.00   None   Yes   None   Sat    None  Dinner   
231 

### Column-wise and multiple function application

## Group-wise operations and transformations

### Apply: General split-apply-combine

#### Suppressing the group keys

### Quantile and bucket analysis

### Example: Filling missing values with group-specific values

## Pivot tables and Cross-tabulation